In [16]:
import pandas as pd
import numpy as np

import os
import pickle
import re
import sys
from IPython.display import display

import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import Dataset, DataLoader

In [17]:
parsed_Qs_file = '../../data/preprocessed/Quantlet/Parsed_Qs_12032023.pkl'

In [8]:
with open(parsed_Qs_file, 'rb') as f:
            parsed_Qs_file = pickle.load(f)

In [9]:
parsed_Qs_file

,folder_name,metainfo_file,type_script,script_name
2,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,02-1_rnd_hd.py
3,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,01-1_rnd_data_processing.py
4,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,04_explore.py
5,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,BitcoinOptionTrading.py
6,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,02-2_rnd_hd_corrected.py
...,...,...,...,...
20387,../../data/QuantLet/UYCTR,"{'Name of Quantlet': 'UYCTReur', 'Published in...",R,LRtest.R
20388,../../data/QuantLet/UYCTR,"{'Name of Quantlet': 'UYCTReur', 'Published in...",R,UYCTReur.R
20389,../../data/QuantLet/UYCTR,"{'Name of Quantlet': 'UYCTReur', 'Published in...",R,Forecastingusd.R
20408,../../data/QuantLet/Metis-Shapley/Additive_Exa...,"{'Name of Quantlet': 'Additive_Example', 'Publ...",ipynb,Additive_Example.ipynb


In [23]:
class QuantletDataset(Dataset):
    def __init__(self, parsed_Qs_file):
        with open(parsed_Qs_file, 'rb') as f:
            self.parsed_Qs_file = pickle.load(f)
        
    def __len__(self):
        return len(self.parsed_Qs_file.script_name)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
    def show_df(self):
        return display(self.parsed_Qs_file) 

In [24]:
dataset = QuantletDataset(parsed_Qs_file)

In [25]:
dataset.show_df()

,folder_name,metainfo_file,type_script,script_name
2,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,02-1_rnd_hd.py
3,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,01-1_rnd_data_processing.py
4,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,04_explore.py
5,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,BitcoinOptionTrading.py
6,../../data/QuantLet/BitcoinOptionTrading/Bitco...,"{'Name of Quantlet': 'BitcoinOptionTrading', '...",py,02-2_rnd_hd_corrected.py
...,...,...,...,...
20387,../../data/QuantLet/UYCTR,"{'Name of Quantlet': 'UYCTReur', 'Published in...",R,LRtest.R
20388,../../data/QuantLet/UYCTR,"{'Name of Quantlet': 'UYCTReur', 'Published in...",R,UYCTReur.R
20389,../../data/QuantLet/UYCTR,"{'Name of Quantlet': 'UYCTReur', 'Published in...",R,Forecastingusd.R
20408,../../data/QuantLet/Metis-Shapley/Additive_Exa...,"{'Name of Quantlet': 'Additive_Example', 'Publ...",ipynb,Additive_Example.ipynb
